In [242]:
import re

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd

# surpress future warning related to SkLearn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from numpy.lib.stride_tricks import sliding_window_view

from sklearn.metrics import classification_report, f1_score, multilabel_confusion_matrix

from sklearn.model_selection import train_test_split


In [540]:
df_r4 = pd.read_csv('../../data_v2/p2p_r4_train_og_mod.csv')
df_r4_il = pd.read_csv('../../data_v2/p2p_r4_il_og_mod.csv')

In [548]:
df_og = pd.read_csv("../../data_v2/hr_extended_features.csv")

In [547]:
df_og

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,frame.number,sniff_time_str,sniff_time,InstanceNumber,synthetic_sniff_time,...,file_data,IsStartActivity,IsEndActivity,ActivityAction,IsEndBP,IsStartBP,selective_filter_data,origin_request_method,origin_selective_filter_data,is_archive_applicant
0,0,0,0,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,26,2020-05-12 09:57:23.762710,2020-05-12 09:57:23.762710,1,2020-05-13 10:07:18.092710,...,['version'],NaN,0.0,Activity Start,False,True,version,NaN,NaN,False
1,1,1,1,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,127,2020-05-12 09:57:28.092427,2020-05-12 09:57:28.092427,1,2020-05-13 22:08:59.592427,...,"['server_version', '12.0-20190820', 'server_ve...",0.0,0.0,NoAction,False,False,NaN,version,version,False
2,2,2,2,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,135,2020-05-12 09:57:28.094494,2020-05-12 09:57:28.094494,1,2020-05-13 22:09:20.264494,...,"['authenticate', 'odoo01', 'dana.wireless@gmai...",0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
3,3,3,3,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,192,2020-05-12 09:57:28.580005,2020-05-12 09:57:28.580005,1,2020-05-13 23:30:15.860005,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
4,4,4,4,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,216,2020-05-12 09:57:28.729685,2020-05-12 09:57:28.729685,1,2020-05-13 23:55:12.809685,...,['6'],0.0,0.0,NoAction,False,False,IsNumber,authenticate,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33068,33068,11958,11958,ResumeReviewActivity_250.pcap,ResumeReviewActivity,729,2020-05-14 00:17:45.346366,2020-05-14 00:17:45.346366,250,2020-05-14 04:15:16.416366,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False
33069,33069,11959,11959,ResumeReviewActivity_250.pcap,ResumeReviewActivity,766,2020-05-14 00:17:45.361497,2020-05-14 00:17:45.361497,250,2020-05-14 04:17:47.741497,...,['1'],0.0,0.0,NoAction,False,False,IsNumber,execute_kw,hr.applicant_write_stage_id_2,True
33070,33070,11960,11960,ResumeReviewActivity_250.pcap,ResumeReviewActivity,774,2020-05-14 00:17:45.363203,2020-05-14 00:17:45.363203,250,2020-05-14 04:18:04.803203,...,"['execute_kw', 'odoo01', '6', '123456789', 'hr...",0.0,0.0,NoAction,False,False,hr.applicant_archive_applicant,NaN,NaN,False
33071,33071,11961,11961,ResumeReviewActivity_250.pcap,ResumeReviewActivity,840,2020-05-14 00:17:45.427437,2020-05-14 00:17:45.427437,250,2020-05-14 04:28:47.207437,...,NaN,0.0,0.0,NoAction,False,False,NaN,NaN,NaN,False


In [535]:
df_r4 = df_r4.fillna("")
df_r4_il = df_r4_il.fillna("")

# Form Sequences by Grouping
df_r4 = df_r4.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_r4["SequenceNumber"] = df_r4.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
df_r4["SequenceNumber"] -= df_r4['SequenceNumber'].min()

In [525]:
df_train = df_r4
df_test = df_r4_il

In [516]:
fw = 10
bw = 5
ws = fw + bw +1

In [539]:
# Reshape Dataframe 

# Fits a Sliding window over the dataframe
def reshape_sliding_window(data, window_shape):
    return np.squeeze(sliding_window_view(data,window_shape),axis =1)

# Reshapes the Dataframe into Sequences of Length N
def reshape_sequences(data, seq_length):
    data = data.values
    num_seq = data.shape[0] // seq_length
    return data[:num_seq*seq_length].reshape((num_seq, seq_length, data.shape[1]))

# Divides dataframe into arrays according to to Seq_Data Indicator
def sequence_by_activities(data, seq_data): 
    return [data[seq_data == i][cols].values for i in range(seq_data.max())]

def chunk_list(list,n): 
    return [list[i * n:(i + 1) * n] for i in range((len(list) + n - 1) // n )]



def extract_labels(data): 
    return data[:,:,-1].tolist()

# Encode Featuees

def seq2features(seq, bw, fw ,feature_list): 
    # seq [0] = Position in Sequence
    # seq [0,4] = Feature 4 an Position 0 
    return [event2features(seq,i, bw, fw, feature_list) for i in range(len(seq))]


def event2features(seq, i,bw, fw, feature_list): 
    # event features 
    features = {
        feature_list[j]: seq[i,j] for j in range(len(feature_list))        
    }
    
    # backward features 1...bw
    for k in range(1,bw+1): 
        index = i - k
        if index >= 0: 
            features.update({
                f"-{k}:{feature_list[j]}" : seq[index,j] for j in range(len(feature_list))
            })
              
    # forward features 1...fw
    for k in range(1, fw+1): 
        index = i + k
        if index < len(seq): 
            features.update({
                f"+{k}:{feature_list[j]}" : seq[index, j] for j in range(len(feature_list))
            })
            
    # remove all emptfey String features        
    features = {key: value for key, value in features.items() if value != ''}
    return features
        


cols = ["event_with_roles", "selective_file_data","origin_file_data","task_position"]
feature_list = ["event_with_roles", "selective_file_data","origin_file_data"]
# Sliding Window 
window_shape = (ws, len(cols))
train_data_sw = reshape_sliding_window(df_train[cols], window_shape)
test_data_sw = reshape_sliding_window(df_test[cols], window_shape)

train_labels_sw = extract_labels(train_data_sw)
test_labels_sw = extract_labels(test_data_sw)

train_features_sw = [seq2features(s, bw, fw, feature_list) for s in train_data_sw]
test_features_sw = [seq2features(s, bw, fw, feature_list) for s in test_data_sw]

# Fixed Sequence Length 
train_data_seq = reshape_sequences(df_train[cols], 15)
test_data_seq = reshape_sequences(df_test[cols], 15)

train_features_seq = [seq2features(s, bw, fw, feature_list) for s in train_data_seq]
test_features_seq = [seq2features(s, bw, fw, feature_list) for s in test_data_seq]

train_labels_seq = extract_labels(train_data_seq)
test_labels_seq = extract_labels(test_data_seq)

# Sequence by Activities
train_data_act = sequence_by_activities(df_train[cols], df_train["SequenceNumber"])
train_labels_act = [seq[:,-1].tolist() for seq in train_data_act]
train_features_act = [seq2features(s, bw, fw, feature_list) for s in train_data_act]

# Generate Features first by overlapping then sequence the data
train_features_ol = chunk_list(seq2features(df_train[feature_list].fillna("").values, bw, fw, feature_list),15)
test_features_ol = chunk_list(seq2features(df_test[feature_list].fillna("").values, bw, fw, feature_list),15)

train_labels_ol = chunk_list(df_train["task_position"].values, 15)
test_labels_ol = chunk_list(df_test["task_position"], 15)

## Model

In [503]:
train_features = train_features_act
train_labels = train_labels_act
test_features = test_features_seq
test_labels = test_labels_seq

In [532]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.3, random_state=42, shuffle=True)

In [534]:
X_train[0]

[{'event_with_roles': 'End Point (Employee)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n]',
  'selective_file_data': 'version',
  '+1:event_with_roles': 'Odoo Application->End Point (Employee): [HttpResponse:HTTP/1.0 200 OK\\r\\n]',
  '+1:origin_file_data': 'version',
  '+2:event_with_roles': 'End Point (Employee)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n]',
  '+3:event_with_roles': "Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['res_users_log']]",
  '+4:event_with_roles': 'Odoo Application->End Point (Employee): [HttpResponse:HTTP/1.0 200 OK\\r\\n]',
  '+4:selective_file_data': 'IsNumber',
  '+5:event_with_roles': 'End Point (Employee)->Odoo Application: [HttpRequest:POST /xmlrpc/2/object HTTP/1.1\\r\\n]',
  '+5:selective_file_data': 'product.product_search_read',
  '+6:event_with_roles': 'Odoo Application->End Point (Employee): [HttpResponse:HTTP/1.0 200 OK\\r\\n]',
  '+6:selective_file_data': 'name',
  

In [530]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=1000, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 425 ms, sys: 33.5 ms, total: 458 ms
Wall time: 647 ms


CRF(algorithm='lbfgs', all_possible_transitions=True, max_iterations=1000)

In [531]:
def flatten(xss): 
    return [x for xs in xss for x in xs]

def evaluate(model, x, y_true): 
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels = model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    print(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred)))
    
evaluate(crf, X_test, y_test)
evaluate(crf, test_features, test_labels) 

0.991114109014183
                  precision    recall  f1-score   support

  position_start       0.97      1.00      0.98       138
position_between       1.00      1.00      1.00      6596
    position_end       1.00      0.98      0.99       121

        accuracy                           1.00      6855
       macro avg       0.99      0.99      0.99      6855
    weighted avg       1.00      1.00      1.00      6855

[[[ 257    2]
  [   5 6591]]

 [[6734    0]
  [   2  119]]

 [[6712    5]
  [   0  138]]]
0.5161416433155563
                  precision    recall  f1-score   support

  position_start       0.88      0.24      0.37        63
position_between       0.97      0.99      0.98      3580
    position_end       0.21      0.18      0.19        62

        accuracy                           0.96      3705
       macro avg       0.69      0.47      0.52      3705
    weighted avg       0.96      0.96      0.96      3705

[[[  33   92]
  [  37 3543]]

 [[3601   42]
  [  51   1

Optimization

In [470]:
np.unique(test_labels_sw)

array(['position_between', 'position_end', 'position_start'], dtype='<U16')

In [471]:
import scipy
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='macro', labels=np.unique(test_labels_sw))

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x2a957dc10>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x32a731850>},
                   scoring=make_scorer(flat_f1_score, average=macro, labels=['position_between' 'position_end' 'position_start']),
                   verbose=1)

In [472]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.23848344457633144, 'c2': 0.012981748133845226}
best CV score: 0.9994634410404596
model size: 0.01M


In [476]:
crf = rs.best_estimator_
evaluate(crf, X_test, y_test)
evaluate(crf, test_features, test_labels)

1.0
                  precision    recall  f1-score   support

  position_start       1.00      1.00      1.00       138
position_between       1.00      1.00      1.00      7148
    position_end       1.00      1.00      1.00       119

        accuracy                           1.00      7405
       macro avg       1.00      1.00      1.00      7405
    weighted avg       1.00      1.00      1.00      7405

[[[ 257    0]
  [   0 7148]]

 [[7286    0]
  [   0  119]]

 [[7267    0]
  [   0  138]]]
0.37837589180050174
                  precision    recall  f1-score   support

  position_start       0.83      0.08      0.14        63
position_between       0.97      0.93      0.95      3580
    position_end       0.02      0.10      0.04        62

        accuracy                           0.91      3705
       macro avg       0.61      0.37      0.38      3705
    weighted avg       0.95      0.91      0.92      3705

[[[  18  107]
  [ 235 3345]]

 [[3402  241]
  [  56    6]]

 [[3641 

In [474]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(9))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-9:])

Top likely transitions:
position_between -> position_end 12.827156
position_between -> position_between 8.457200
position_start -> position_between 5.971737
position_between -> position_start -0.665763
position_start -> position_start -2.707565
position_end -> position_start -4.859459
position_end -> position_between -6.631930

Top unlikely transitions:
position_between -> position_end 12.827156
position_between -> position_between 8.457200
position_start -> position_between 5.971737
position_between -> position_start -0.665763
position_start -> position_start -2.707565
position_end -> position_start -4.859459
position_end -> position_between -6.631930


In [475]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
4.299834 position_start event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['fetchmail_server']]
1.868129 position_start selective_file_data:version
1.868129 position_start +1:origin_file_data:version
1.139334 position_between -1:event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['fetchmail_server']]
0.855205 position_start +3:event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['res_users_log']]
0.727161 position_start +1:event_with_roles:End Point (Procurement)->Odoo Application: [HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n]
0.357684 position_end -1:event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_message']]
0.301963 position_end selective_file_data:IsNumber
0.234932 position_start +2:event_with_roles:Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['res_users_log']]
0.2